# Covid-19 New Case Predictor 
### Using Neural Network with four input: date, state, newCases,  newDeaths
-----------------------------------------------
by Roya Salei

Course CS4200- Artificial Intelligence 

Professor: Dr. Husain Zaidi

In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [15]:
# read data from file
df = pd.read_csv("state_ts.csv")
df.tail(10)


,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,hospitalBeds.capacity,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
33027,2021-11-02,WY,367.0,103989.0,69.0,1243.0,81211.0,1021342.0,255199.0,1603.0,818.0,195.0,139.0,74.0,42.0,0,0.636213,0.119615,3.779609,0.710608
33028,2021-11-03,WY,414.0,104403.0,0.0,1243.0,81551.0,1025572.0,255597.0,1748.0,939.0,192.0,139.0,76.0,41.0,0,0.717690,0.000000,4.263646,0.000000
33029,2021-11-04,WY,537.0,104940.0,0.0,1243.0,81819.0,1028372.0,255655.0,1748.0,941.0,179.0,139.0,72.0,36.0,0,0.930916,0.000000,5.530381,0.000000
33030,2021-11-05,WY,378.0,105318.0,0.0,1243.0,82057.0,1030728.0,256237.0,1748.0,923.0,181.0,139.0,76.0,36.0,0,0.655282,0.000000,3.892894,0.000000
33031,2021-11-06,WY,0.0,105318.0,0.0,1243.0,82202.0,1032878.0,256695.0,1747.0,881.0,169.0,137.0,79.0,37.0,0,0.000000,0.000000,0.000000,0.000000
33032,2021-11-07,WY,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,1747.0,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
33033,2021-11-08,WY,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,1602.0,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000
33034,2021-11-09,WY,297.0,106287.0,55.0,1298.0,NaN,NaN,257742.0,1603.0,831.0,181.0,137.0,81.0,39.0,0,0.514864,0.095345,3.058702,0.566426
33035,2021-11-10,WY,411.0,106698.0,0.0,1298.0,NaN,NaN,258081.0,1748.0,963.0,174.0,136.0,77.0,36.0,0,0.712489,0.000000,4.232750,0.000000
33036,2021-11-11,WY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1748.0,963.0,174.0,136.0,77.0,36.0,0,NaN,NaN,NaN,NaN


In [16]:
# change state name to integer
states = df['state'].unique()

for i, j in enumerate(states):
    df['state'] = df['state'].replace(j, i)   
df

,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,hospitalBeds.capacity,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
0,2020-03-01,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,2020-03-02,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2,2020-03-03,0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,2020-03-04,0,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,2020-03-05,0,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33032,2021-11-07,52,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,1747.0,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
33033,2021-11-08,52,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,1602.0,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000
33034,2021-11-09,52,297.0,106287.0,55.0,1298.0,NaN,NaN,257742.0,1603.0,831.0,181.0,137.0,81.0,39.0,0,0.514864,0.095345,3.058702,0.566426
33035,2021-11-10,52,411.0,106698.0,0.0,1298.0,NaN,NaN,258081.0,1748.0,963.0,174.0,136.0,77.0,36.0,0,0.712489,0.000000,4.232750,0.000000


In [17]:
# convert date column to float type
df['date'] = pd.to_datetime(df['date'])

for i in range(len(df)):
    df['date'][i]=df['date'][i].timestamp()

# df.shape
df

c:\users\jooni\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,hospitalBeds.capacity,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
0,1.58302e+09,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,1.58311e+09,0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2,1.58319e+09,0,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,1.58328e+09,0,NaN,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,1.58337e+09,0,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33032,1.63624e+09,52,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,1747.0,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
33033,1.63633e+09,52,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,1602.0,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000
33034,1.63642e+09,52,297.0,106287.0,55.0,1298.0,NaN,NaN,257742.0,1603.0,831.0,181.0,137.0,81.0,39.0,0,0.514864,0.095345,3.058702,0.566426
33035,1.6365e+09,52,411.0,106698.0,0.0,1298.0,NaN,NaN,258081.0,1748.0,963.0,174.0,136.0,77.0,36.0,0,0.712489,0.000000,4.232750,0.000000


In [18]:
#Drop rows that has any NaN data
df = df.dropna()
df = df.reset_index()
df.tail(10)

,index,date,state,newCases,cases,newDeaths,deaths,positiveTests,negativeTests,vaccinationsCompleted,...,hospitalBeds.currentUsageTotal,hospitalBeds.currentUsageCovid,icuBeds.capacity,icuBeds.currentUsageTotal,icuBeds.currentUsageCovid,mask_mandate,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
14532,33024,1.63555e+09,52,0.0,102926.0,0.0,1174.0,80690.0,1017147.0,253968.0,...,892.0,193.0,136.0,79.0,47.0,0,0.000000,0.000000,0.000000,0.000000
14533,33025,1.63564e+09,52,0.0,102926.0,0.0,1174.0,80820.0,1017853.0,254028.0,...,864.0,188.0,137.0,76.0,43.0,0,0.000000,0.000000,0.000000,0.000000
14534,33026,1.63572e+09,52,696.0,103622.0,0.0,1174.0,81032.0,1019225.0,254047.0,...,782.0,195.0,138.0,71.0,45.0,0,1.206551,0.000000,7.167868,0.000000
14535,33027,1.63581e+09,52,367.0,103989.0,69.0,1243.0,81211.0,1021342.0,255199.0,...,818.0,195.0,139.0,74.0,42.0,0,0.636213,0.119615,3.779609,0.710608
14536,33028,1.6359e+09,52,414.0,104403.0,0.0,1243.0,81551.0,1025572.0,255597.0,...,939.0,192.0,139.0,76.0,41.0,0,0.717690,0.000000,4.263646,0.000000
14537,33029,1.63598e+09,52,537.0,104940.0,0.0,1243.0,81819.0,1028372.0,255655.0,...,941.0,179.0,139.0,72.0,36.0,0,0.930916,0.000000,5.530381,0.000000
14538,33030,1.63607e+09,52,378.0,105318.0,0.0,1243.0,82057.0,1030728.0,256237.0,...,923.0,181.0,139.0,76.0,36.0,0,0.655282,0.000000,3.892894,0.000000
14539,33031,1.63616e+09,52,0.0,105318.0,0.0,1243.0,82202.0,1032878.0,256695.0,...,881.0,169.0,137.0,79.0,37.0,0,0.000000,0.000000,0.000000,0.000000
14540,33032,1.63624e+09,52,0.0,105318.0,0.0,1243.0,82344.0,1033805.0,256745.0,...,881.0,175.0,137.0,78.0,41.0,0,0.000000,0.000000,0.000000,0.000000
14541,33033,1.63633e+09,52,672.0,105990.0,0.0,1243.0,82367.0,1034107.0,256772.0,...,806.0,185.0,139.0,74.0,41.0,0,1.164946,0.000000,6.920700,0.000000


In [19]:
# select mask mandate as a label for categorizing data
label = df['newCases'].values
label.shape

(14542,)

In [20]:
# mask_as_label

In [21]:
# select 4 columns for evidence variables
# data = df[['date','state','newCases', 'newDeaths', 'positiveTests','vaccinationsCompleted', 'daily_infection_rate', 'daily_death_rate',
#        'newCases_density', 'newDeaths_density']]

mini_df = df[['date','state', 'newDeaths', 'positiveTests','vaccinationsCompleted', 'daily_infection_rate', 
              'daily_death_rate','newCases_density', 'newDeaths_density']]
mini_df.shape
mini_df

,date,state,newDeaths,positiveTests,vaccinationsCompleted,daily_infection_rate,daily_death_rate,newCases_density,newDeaths_density
0,1.61058e+09,0,2.0,89541.0,7013.0,0.456782,0.002727,0.585715,0.003497
1,1.61067e+09,0,1.0,89966.0,9406.0,0.411786,0.001364,0.528017,0.001748
2,1.61101e+09,0,1.0,91044.0,10802.0,0.175895,0.001364,0.225544,0.001748
3,1.6111e+09,0,23.0,91320.0,11283.0,0.227709,0.031361,0.291983,0.040213
4,1.61119e+09,0,1.0,91607.0,12352.0,0.274069,0.001364,0.351429,0.001748
...,...,...,...,...,...,...,...,...,...
14537,1.63598e+09,52,0.0,81819.0,255655.0,0.930916,0.000000,5.530381,0.000000
14538,1.63607e+09,52,0.0,82057.0,256237.0,0.655282,0.000000,3.892894,0.000000
14539,1.63616e+09,52,0.0,82202.0,256695.0,0.000000,0.000000,0.000000,0.000000
14540,1.63624e+09,52,0.0,82344.0,256745.0,0.000000,0.000000,0.000000,0.000000


In [22]:
# convert evidence to numpy array and check date type- it should be float
evidence = mini_df.to_numpy()
evidence = np.asarray(evidence).astype('float32')

type(evidence[0][0])  # check type of evidence to make sure it is all float32

numpy.float32

In [23]:
# evidence.shape
# evidence

### Five Steps in Training Life-Cycle 
1. Define the model

In [24]:
# Separate data into training and testing groups
X_training, X_testing, y_training, y_testing = train_test_split(
    evidence, label, test_size=0.4
)

In [25]:
# Create a neural network - meaning one layer after another layer
model = tf.keras.models.Sequential()

In [34]:
# Add a hidden layer with 8 units, with ReLU activation
# Dense layer means each node is densly connected to other layers
model.add(tf.keras.layers.Dense(8, input_shape=(9,), activation="relu"))

In [35]:
# Add output layer with 1 unit, with sigmoid activation
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

2. Compile the model

In [36]:
# Train neural network
model.compile(
    optimizer="adam",  # choice of optimization
    loss="binary_crossentropy",  # what type of loss function
    metrics=["accuracy"] # the metrics to analyze for
)

In [37]:
print(X_training.shape, X_testing.shape, y_training.shape, y_testing.shape) 

(8725, 9) (5817, 9) (8725,) (5817,)


3. Fit or Train the Model

In [39]:
# train the model for 20 times
model.fit(X_training, y_training, epochs=20)

Epoch 1/20
273/273 [==============================] - 1s 5ms/step - loss: -159456.4219 - accuracy: 3.4384e-04
Epoch 2/20
273/273 [==============================] - 1s 5ms/step - loss: -172788.5469 - accuracy: 3.4384e-04
Epoch 3/20
273/273 [==============================] - 1s 5ms/step - loss: -186560.6094 - accuracy: 3.4384e-04
Epoch 4/20
273/273 [==============================] - 1s 5ms/step - loss: -200779.4219 - accuracy: 3.4384e-04
Epoch 5/20
273/273 [==============================] - 1s 5ms/step - loss: -215428.9062 - accuracy: 3.4384e-04
Epoch 6/20
273/273 [==============================] - 1s 5ms/step - loss: -230518.5781 - accuracy: 3.4384e-04
Epoch 7/20
273/273 [==============================] - 1s 5ms/step - loss: -246113.5469 - accuracy: 3.4384e-04
Epoch 8/20
273/273 [==============================] - 1s 5ms/step - loss: -262140.7969 - accuracy: 3.4384e-04
Epoch 9/20
273/273 [==============================] - 1s 5ms/step - loss: -278575.0938 - accuracy: 3.4384e-04
Epoch 10/2

4. Evaluate the Model

In [40]:
# Evaluate and compare with test data:  how well model performs
model.evaluate(X_testing, y_testing, verbose=2)

182/182 - 0s - loss: -5.0467e+05 - accuracy: 8.5955e-04


[-504673.34375, 0.0008595496183261275]

5. Make Predictions

In [41]:
#predict
d1 =[1.63616e+09, 52  ,0.0 ,82202.0 ,256695.0 ,0.000000 ,0.000000 ,0.000000 ,0.000000 ]
yhat = model.predict([d2])
print('Predicted: %.3f' % yhat)

NameError: name 'd2' is not defined